<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [1]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code of the response:

In [3]:
response.headers['Content-Type']

'application/json'

We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

In [4]:
parameters = {"lat": -33.87, "lon": 151.21}

The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [5]:
response = requests.get("http://api.open-notify.org/iss-pass.json", params = parameters)

Print the response header:

In [6]:
print(response.headers)

{'Server': 'nginx/1.10.3', 'Date': 'Tue, 11 Jan 2022 09:01:35 GMT', 'Content-Type': 'application/json', 'Content-Length': '521', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}


Print the content of the response (the data that the server returned):

In [7]:
print(response.content)

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1641891695, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 626, \n      "risetime": 1641937803\n    }, \n    {\n      "duration": 629, \n      "risetime": 1641943605\n    }, \n    {\n      "duration": 445, \n      "risetime": 1641949562\n    }, \n    {\n      "duration": 351, \n      "risetime": 1641955519\n    }, \n    {\n      "duration": 533, \n      "risetime": 1641961323\n    }\n  ]\n}\n'


Note that this is a Python byte string:

In [8]:
print(type(response.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [9]:
print(response.headers["content-type"])

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [10]:
overheads = response.json()
print(overheads)

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1641891695, 'latitude': -33.87, 'longitude': 151.21, 'passes': 5}, 'response': [{'duration': 626, 'risetime': 1641937803}, {'duration': 629, 'risetime': 1641943605}, {'duration': 445, 'risetime': 1641949562}, {'duration': 351, 'risetime': 1641955519}, {'duration': 533, 'risetime': 1641961323}]}


What kind of object did this give us?

In [11]:
print(type(overheads))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `response` value to a separate dict:

In [12]:
passes = overheads['response']

In [13]:
passes[1]['risetime']

1641943605

In [14]:
x = []
for item in passes:
    x.append(item['risetime'])
x

[1641937803, 1641943605, 1641949562, 1641955519, 1641961323]

Now extract the `risetime` strings into an array called `srisetimes`:

In [15]:
srisetimes = [xpass['risetime'] for xpass in passes]
srisetimes

[1641937803, 1641943605, 1641949562, 1641955519, 1641961323]

```
datetime.fromtimestamp(str)
```

Convert these to an array of Python `datetime` values called `risetimes`:

In [16]:
risetimes = [datetime.fromtimestamp(xpass['risetime']) for xpass in passes] 

In [17]:
risetimes

[datetime.datetime(2022, 1, 12, 7, 50, 3),
 datetime.datetime(2022, 1, 12, 9, 26, 45),
 datetime.datetime(2022, 1, 12, 11, 6, 2),
 datetime.datetime(2022, 1, 12, 12, 45, 19),
 datetime.datetime(2022, 1, 12, 14, 22, 3)]

Finally, print these in a format that people understand:



```
str.strftime('%d/%m/%y %I:%M')

27/03/19 10:59
27/03/19 12:35
27/03/19 02:11
28/03/19 03:42
28/03/19 05:13
```



In [18]:
for risetime in risetimes:
    print(risetime.strftime('%d/%m/%y %I:%M'))

12/01/22 07:50
12/01/22 09:26
12/01/22 11:06
12/01/22 12:45
12/01/22 02:22


Here is an endpoint that tells us who is onboard:

In [20]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [21]:
astros = response.json()
print(astros)
print(astros["number"])
[print(astronaut['name']) for astronaut in astros['people']]

{'people': [{'craft': 'ISS', 'name': 'Mark Vande Hei'}, {'craft': 'ISS', 'name': 'Pyotr Dubrov'}, {'craft': 'ISS', 'name': 'Anton Shkaplerov'}, {'craft': 'Shenzhou 13', 'name': 'Zhai Zhigang'}, {'craft': 'Shenzhou 13', 'name': 'Wang Yaping'}, {'craft': 'Shenzhou 13', 'name': 'Ye Guangfu'}, {'craft': 'ISS', 'name': 'Raja Chari'}, {'craft': 'ISS', 'name': 'Tom Marshburn'}, {'craft': 'ISS', 'name': 'Kayla Barron'}, {'craft': 'ISS', 'name': 'Matthias Maurer'}], 'message': 'success', 'number': 10}
10
Mark Vande Hei
Pyotr Dubrov
Anton Shkaplerov
Zhai Zhigang
Wang Yaping
Ye Guangfu
Raja Chari
Tom Marshburn
Kayla Barron
Matthias Maurer


[None, None, None, None, None, None, None, None, None, None]

## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [22]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code, 
    '''
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

2. Test your response handler on some correct and incorrect API calls.

In [23]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    
response = requests.get("http://api.open-notify.org/iss-pass.json")
handleResponse(response, True)[0]

TypeError: 'NoneType' object is not subscriptable

>

>

>



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



